# Residential Building Classification
This is a bare-bone version of the residential vs. non-residential building classification. We use DenseNet in this implementation. 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.utils.data import TensorDataset

import numpy as np

from cs231n.data_utils import *


## Preprocess FMOW Dataset
Check if fMoW dataset is available. If so, process the images by cropping based on given bounding boxes. Load in traing, validation and test dataset along with ground truth labels for training and validation set. 

In [ ]:
class_names = ['false_detection','residential','non_residential']
dtype = torch.FloatTensor

data = load_fmow('~/ext/data/fmow-rgb', dtype=dtype)
# Unpack training and validation dat
X_train = torch.from_numpy(data['X_train'])
y_train = torch.from_numpy(data['y_train'])
train_data = data_utils.TensorDataset(X_train, y_train)
loader_train = DataLoader(train_loader, batch_size=64)

X_val = torch.from_numpy(data['X_val'])
y_val = torch.from_numpy(data['y_val'])
val_data = data_utils.TensorDataset(X_val, y_val)
loader_val = DataLoader(val_data, batch_size=64)

# Unpack test data
X_test = torch.from_numpy(data['X_test'])
y_test = torch.from_numpy(data['y_test'])
test_data = data_utils.TensorDataset(X_test, y_test)
loader_test = DataLoader(test_data, batch_size=64)

# Print out all the keys and values from the data dictionary
for k, v in data.items():
    print(k, type(v), v.shape, v.dtype)


### Visualize a small batch of data
Plot a small batch of training images to get a sense of data

In [ ]:
# Sample a minibatch and show the images and captions
batch_size = 5

X_mini, y_mini = sample_mini_batch(data['X_train'], data['y_train'], batch_size=batch_size)
for i, y_i in enumerate(y_mini):
    plt.subplot(1, batch_size, i+1)
    plt.imshow(X_mini[i, :, :, :])
    plt.axis('off')
    plt.title(class_names[y_mini[i]])
    plt.show()

## Load Model
Load in pre-trained DenseNet. Configure training parameters. 

### Naive two-layer CNN

In [ ]:
# CNN Model (3 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=6, stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(12*12*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
#     200*200  
#     98*98
#     49*49
#     24*24
#     12*12
#     11111---1

In [ ]:
# Params
learning_rate = 1e-4
num_epochs = 50
print_every = 100

model = CNN().type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

## Train Model
Train the model one epoch at a time. Keep the verbose option on to monitor the loss during training. 
Save training and validation accuracy history for visualization. 

In [ ]:
# Define Train function
def train(model, loss_fn, optimizer, loader, epochs=1):
    for epoch in range(epochs):
        # Set on Training mode
        model.train()
        for t, (x, y) in enumerate(loader):
            x_var = Variable(x.type(dtype))
            y_var = Variable(y.type(dtype))
            socres = model(x_var)
            loss = loss_fn(scores, y_var)
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
def check_accuracy(model, loader):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in loader:
        x_var = Variable(x.type(dtype), volatile=True)
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:
# Train the model
train(model, loss_fn, optimizer, loader_train, num_epochs=num_epochs)
# Check accuracy of model
check_accuracy(model, loader_val)

## Visualization
Visualize training and validation accuracy throughout training history. 
Visualize loss function. 

## Test Model
Make predictions based on test dataset. Visually assess the quality of prediction and compute accuracy based on groundtruth. 

In [ ]:
# Check accuracy of model on Test set
check_accuracy(model, loader_test)